In [4]:
from absbox import API, mkDeal
import pandas as pd 
localAPI = API("http://localhost:8081/",check=False, lang="english")


name = "Deal"
dates = {
    "cutoff":"2024-08-31","closing":"2024-08-31","firstPay":"2024-09-30",
    "stated": "2027-09-30",
    "poolFreq": "MonthEnd",
    "payFreq": "MonthEnd"
}

amortising_loan = ["Mortgage"
            ,{"originBalance": 1092.5
            ,"originRate": ["fix",0.22]
            ,"originTerm": 11
            ,"freq": "Monthly"
            ,"type": "level"
            ,"originDate": "2024-09-01"}
            ,{"currentBalance": 1092.5
            ,"currentRate": 0.22
            ,"remainTerm": 11
            ,"status": "current"}]

bullet_loans = ["Loan"
            ,{"originBalance": 5750
            ,"originRate": ["fix",0.22]
            ,"originTerm": 3
            ,"freq": "Monthly"
            ,"type": "i_p"
            ,"originDate": "2024-09-01"}
            ,{"currentBalance": 5750
            ,"currentRate": 0.22
            ,"remainTerm": 3
            ,"status": "current"}]

j = {"pool_amortizing": {'assets':[amortising_loan]*1000},"pool_bullet": {'assets':[bullet_loans]*10}
     }

pool = {
    "name": "name",
    "assets": j,
    "issuanceStat": {"IssuanceBalance": 1_150_000.00},
}


accounts = {
    "collections_interest": {"balance": 0},
    "collections_principal": {"balance": 0},
    "revolBuyAcc": {"balance": 0},
}
bonds = {
    "A": {"A1": {
        "balance": 920_000.84,
        "rate": 0.13,
        "originBalance": 10_000_000,
        "originRate": 0.13,
        "startDate": "2024-08-31",
        "rateType": {"fix": 0.13, "dayCount": "DC_ACT_365A"},
        "maturityDate": "2027-11-30",
        "bondType": {"Lockout": "2026-11-30"},
    }},
    "EQ": {
        "balance": 1.0,
        "rate": 0.0,
        "originBalance": 1_000_000,
        "originRate": 0.0,
        "startDate": "2024-08-31",
        "maturityDate": "2027-11-30",
        "rateType": {"Fixed": 0.00},
        "bondType": {"Equity": None},
    },
}
                                

fees = None
waterfall = {
                    "Accelerated":[
                                ["calcIntByGroup", ["A"]],
                                ["fundWith", "revolBuyAcc", "A1",{"balCapAmt": 500_000}],
                                ["accrueAndPayIntByGroup", "collections_interest", "A","byStartDate"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc",{
                                "formula": ("-",("bondBalance", "A1"),("factor",("bondBalance", "A1"),0.75,)
                                            ,)
                                            ,}
                                    ,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc",{
                                "formula": ("-",("bondBalance", "A1"),("factor",("bondBalance", "A1"),0.05,)
                                            ,)
                                            ,}
                                    ,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ",{"support":["account","collections_principal"]}],
                                ],
                    "revolving":[
                                ["calcIntByGroup", ["A"]],
                                ["accrueAndPayIntByGroup", "collections_interest", "A","byStartDate"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc",{
                                "formula": ("-",("bondBalance", "A1"),("factor",("bondBalance", "A1"),0.75,)
                                            ,)
                                            ,}
                                    ,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc",{
                                "formula": ("-",("bondBalance", "A1"),("factor",("bondBalance", "A1"),0.05,)
                                            ,)
                                            ,}
                                    ,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ",{"support":["account","collections_principal"]}],
                                ],
                    "amortizing":[
                                ["calcIntByGroup", ["A"]],  # actions if deal is in Amortizing status
                                ["accrueAndPayIntByGroup", "collections_interest", "A","byStartDate",{"support":["account","collections_principal"]}],
                                ["payPrinByGroup","collections_principal","A","byStartDate"],
                                ["payIntResidual", "collections_principal", "EQ"],
                                ["payPrin", "collections_principal", ["EQ"]],],
                    "cleanUp": [
                        ["sellAsset", ["Current|Defaulted", 1.0, 0], "collections_principal"],
                        ["accrueAndPayIntByGroup", "collections_interest", "A","byStartDate",{"support":["account","collections_principal"]}],
                        ["payPrinByGroup","collections_principal","A","byStartDate"],
                        ["payIntResidual", "collections_principal", "EQ"],
                        ["payPrin", "collections_principal", ["EQ"]],
                    ],
                }

collects = [
            [["pool_amortizing"],"CollectedPrincipal", "collections_principal"],
            [["pool_amortizing"],"CollectedRecoveries", "collections_principal"],
            [["pool_amortizing"],"CollectedPrepayment", "collections_principal"],
            [["pool_amortizing"],"CollectedInterest", "collections_interest"],
            [["pool_bullet"],"CollectedPrincipal", "collections_principal"],
            [["pool_bullet"],"CollectedRecoveries", "collections_principal"],
            [["pool_bullet"],"CollectedPrepayment", "collections_principal"],
            [["pool_bullet"],"CollectedInterest", "collections_interest"],
        ]


trigger = {
            "BeforeDistribution": {
                "revolving_trigger": {
                    "condition": ["any", [">=", "2025-12-31"]],
                    "effects": ("newStatus", "Revolving"),
                    "status": False,
                    "curable": False,
                },
                "amortization_trigger": {
                    "condition": ["any", [">=", "2026-11-30"]],
                    "effects": ("newStatus", "Amortizing"),
                    "status": False,
                    "curable": False,
                }
            }
        }

deal_data = {
            "name": name,
            "dates": dates,
            "pool": j,
            "accounts": accounts,
            # "fees": fees,
            "bonds": bonds,
            "waterfall": waterfall,
            "collect": collects,
            "trigger": trigger,
            "status": ("PreClosing","Accelerated"),
        }
deal = mkDeal(deal_data)
revol_asset1 = amortising_loan
revol_asset2 = bullet_loans


revolvingPool1 = (["constant",revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":0.0},None,None,None)
                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Loan",{"CDR":0.0},None,None,None)
                   ,None
                   ,None))

r = localAPI.run(deal,
                #  poolAssump = ("ByPoolId",
                #                     {"pool_amortizing":amortisingPoolAssump
                #                     ,"pool_bullet":bulletPoolAssump}
                #                     )
         runAssump=[("revolving"
                       ,{"Pool1":revolvingPool1,"Pool2":revolvingPool2})]
         ,read=True)


Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.4

Warning Message from server:
A1 not in fromList ["A","EQ"] Or revolBuyAcc not in fromList 
["collections_interest","collections_principal","revolBuyAcc"]

TypeError: 'NoneType' object is not subscriptable

In [5]:
r['bonds']['A']

{'A1':               balance  interest  principal  rate       cash    intDue  \
 date                                                                    
 2024-09-30  920000.84      0.00       0.00  0.13       0.00   9830.14   
 2024-10-31  920000.84  19987.95       0.00  0.13   19987.95      0.00   
 2024-11-30  920000.84   9830.14       0.00  0.13    9830.14      0.00   
 2024-12-31  920000.84  10157.81       0.00  0.13   10157.81      0.00   
 2025-01-31  920000.84  10157.81       0.00  0.13   10157.81      0.00   
 2025-02-28  920000.84   9174.80       0.00  0.13    9174.80      0.00   
 2025-03-31  920000.84  10157.81       0.00  0.13   10157.81      0.00   
 2025-04-30  920000.84   9830.14       0.00  0.13    9830.14      0.00   
 2025-05-31  920000.84  10157.81       0.00  0.13   10157.81      0.00   
 2025-06-30  920000.84   9830.14       0.00  0.13    9830.14      0.00   
 2025-07-31  920000.84  10157.81       0.00  0.13   10157.81      0.00   
 2025-08-31  920000.84  10157.81